In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_excel("data.xlsx")
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019,"Chicago, Illinois, USA",Red,False,Lightweight,3,0,...,3,6,1,14,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0,...,1,0,0,6,Orthodox,162.56,172.72,135.0,26.0,29.0
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0


### Apenas a categoria Peso-Pesados ('Heavyweights')

In [4]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [5]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

### Blue = 0
### Red = 1

In [6]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)

In [11]:
data_heavy_util.head(2)

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [27]:
colunas = list(data_heavy_util)

In [28]:
colunas.remove('R_fighter')
colunas.remove('B_fighter')
colunas.remove('title_bout')
colunas.remove('weight_class')
colunas.remove('no_of_rounds')
colunas.remove('B_total_rounds_fought')
colunas.remove('R_total_rounds_fought')

In [11]:
dataf = data_heavy_util.loc[:,colunas]
dataf.head()

,Winner,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,0,0,1,0,17.0,14.5,2.5,2.0,201.0,59.5,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,1,0,1,0,8.0,8.0,15.0,13.0,85.0,38.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,1,2,0,0,5.0,3.5,5.5,3.5,16.0,5.5,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,0,0,1,0,35.0,21.0,38.0,30.0,167.0,69.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [11]:
x=colunas
y=x


In [ ]:
sns.set(style="ticks")
sns.pairplot(data_heavy_util, hue="origin", x_vars=x, y_vars=y)

### Correlações

In [14]:
data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)

In [15]:
blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head(2)

,date,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
date,1.000000,-0.161123,-0.032762,0.054294,-0.145747,NaN,0.274744,0.256441,0.154872,0.179436,...,0.231107,0.255263,-0.078268,-0.051332,0.205434,0.029779,0.010070,0.050460,0.102656,0.258130
title_bout,-0.161123,1.000000,0.491461,-0.137166,0.345926,NaN,0.000122,0.002365,0.059811,0.056089,...,0.198632,0.182796,0.064792,-0.056728,0.181773,-0.018567,0.100335,-0.185558,0.014377,-0.040564


### Aplicando filtros

In [18]:
filtros = ['R_fighter','B_fighter','Winner','weight_class','title_bout','R_wins','R_losses','B_wins','B_losses',
           'B_Reach_cms','B_Height_cms','B_Weight_lbs','R_Reach_cms','R_Height_cms','R_Weight_lbs']

In [19]:
data_heavy_filtered = data_heavy.loc[:,filtros]

In [22]:
data_heavy_filtered.head()

,R_fighter,B_fighter,Winner,weight_class,title_bout,R_wins,R_losses,B_wins,B_losses,B_Reach_cms,B_Height_cms,B_Weight_lbs,R_Reach_cms,R_Height_cms,R_Weight_lbs
4,Tai Tuivasa,Blagoy Ivanov,0,Heavyweight,0,3,1,1,1,185.42,180.34,250.0,190.50,187.96,264.0
54,Walt Harris,Serghei Spivac,1,Heavyweight,0,4,7,0,0,198.12,190.50,260.0,195.58,195.58,250.0
60,Arjan Bhullar,Juan Adams,1,Heavyweight,0,2,1,1,0,203.20,195.58,265.0,190.50,185.42,245.0
63,Greg Hardy,Dmitrii Smoliakov,1,Heavyweight,0,0,1,0,2,187.96,187.96,253.0,203.20,195.58,265.0
69,Andrei Arlovski,Augusto Sakai,0,Heavyweight,0,16,12,1,0,195.58,190.50,265.0,195.58,190.50,240.0


In [42]:
data_heavy_filtered['B_Reach_cms'].corr(data_heavy_filtered['Winner'])

-0.1090621360814943

In [56]:
data_heavy_util=data_heavy_util.dropna()
for nome_col in colunas:
    #c=data_heavy_util[colunas[i]].corr(data_heavy_util['Winner'])
    c=data_heavy_util[nome_col].corr(data_heavy_util['Winner'])
    print(c)
    if c<0.05 and c>0:
        data_heavy_util.drop(nome_col,axis=1)  
    elif  c<0 and c>-0.05:
        data_heavy_util.drop(nome_col,axis=1)


1.0
-0.027342948953226653
-0.04828682669155675
nan
-0.16603794298259608
-0.16559627693779944
-0.0697719647024303
-0.06763805063917308
-0.1463959996954793
-0.12671746572970893
-0.016733414582464478
-0.026687437548784727
-0.1453054067778058
-0.11133382962779527
0.07122980292334141
-0.020577844342641788
-0.020490664663967716
0.04017253694796535
0.1214035354297481
-0.1550384078186563
-0.1346367476122478
-0.08843797172482522
0.06194430584400583
-0.04206378201932684
-0.027842599636433603
0.029911312911948335
-0.13953164008759802
-0.10517653252438368
-0.019017127976364498
0.07740828475410316
-0.06247875663489411
-0.04077235673989077
-0.029993908422933563
-0.018489961877842243
-0.07991477644154811
-0.052751962060009955
0.08146636096288215
0.08366109134199903
-0.054858780721967364
-0.011120779001125129
0.08563158143826338
-0.037321285650113785
-0.013319848497432151
0.057591172903410495
0.05113637144960403
-0.06311866826629027
-0.02381251002248239
0.01877116466321957
0.016581698915309508
-0.0218

TypeError: unsupported operand type(s) for /: 'str' and 'int'